In [1]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [2]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [3]:
includet("utils.jl")
includet("turing_nw_bvar.jl")

In [4]:
df = DataFrame(XLSX.readtable("../../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

146×15 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-03-01 to 2024-04-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1988-03-01 │   0.0857 │   677.401 │       262.6 │  9.04333 │  18.59 │ 14.433 ⋯
│ 1988-06-01 │   0.0882 │   681.206 │       269.1 │  9.30667 │  21.67 │ 14.608 ⋯
│ 1988-09-01 │   0.0887 │   694.387 │       277.4 │     9.55 │  22.73 │ 14.811 ⋯
│ 1988-12-01 │   0.0914 │   697.568 │       285.4 │  9.81333 │  23.75 │ 15.088 ⋯
│ 1989-03-01 │    0.093 │   706.377 │       302.3 │  10.0867 │  24.96 │ 15.686 ⋯
│ 1989-06-01 │    0.081 │    780.73 │       331.9 │  10.4233 │  25.22 │ 17.013 ⋯
│ 1989-09-01 │   0.0831 │   785.343 │       347.4 │  10.7967 │  23.69 │ 17.640 ⋯
│ 1989-12-01 │   0.0793 │   821.683 │      339.97 │    11.14 │  22.87 │ 17.048 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [27]:
freq = 4
returns = percentchange(data_source[:ACWI_Net, :USA_Net, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
real_10Y = log1p.(data_source[:Yeld_10Y])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend_Yeld])
term = log1p.(data_source[:Yeld_10Y])  .- log1p.(data_source[:T_Bill])
#data = merge(real_returns, real_t_bill, term, cpi, dy)
data = real_returns

start = Date(1989, 01,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net,0.1527,1.1429,0.068,0.2358,-1.6892,-1.3081,-0.0621,0.1256,7.5547,0.4454,0.1125
USA_Net,0.1466,0.9025,0.096,0.283,-1.4498,-1.2239,-0.0302,0.131,5.1481,0.6551,0.1176
Price_10Y,0.0836,0.5691,0.0533,0.1694,-0.2899,0.3136,-0.0702,0.0162,0.0266,0.6377,0.0313
World_ex_USA,0.1676,1.2168,0.0465,0.2624,-1.7796,-0.9618,-0.1504,0.124,5.7703,0.2772,0.0755
EM_Net,0.253,1.7838,0.0802,0.3814,-2.3616,-0.8653,-0.1935,0.0758,4.3012,0.3168,0.1287


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net_CPI,0.1499,1.1035,0.0413,0.2046,-1.5876,-1.0841,-0.085,0.1166,6.0996,0.2756,0.0671
USA_Net_CPI,0.1449,0.8631,0.0693,0.2568,-1.3483,-1.0427,-0.0548,0.1149,3.9088,0.4783,0.0943
Price_10Y_CPI,0.0893,0.6707,0.0266,0.1468,-0.3841,0.3686,-0.1026,0.0313,0.5634,0.2975,0.0121
World_ex_USA_CPI,0.1641,1.1774,0.0197,0.226,-1.678,-0.7637,-0.1656,0.1193,4.7076,0.1204,0.0465
EM_Net_CPI,0.2492,1.7444,0.0534,0.3523,-2.26,-0.765,-0.2243,0.0713,3.8365,0.2144,0.0663


...

"Excess_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net_T_Bill,0.1529,1.1411,0.0407,0.2043,-1.6895,-1.1946,-0.0963,0.1298,7.1313,0.266,0.0729
USA_Net_T_Bill,0.146,0.9007,0.0687,0.2387,-1.4501,-1.1617,-0.0599,0.1267,4.9038,0.4703,0.0939
Price_10Y_T_Bill,0.0827,0.5688,0.0259,0.1303,-0.2919,0.3958,-0.0921,-0.0094,0.2379,0.3139,0.0181
World_ex_USA_T_Bill,0.1679,1.215,0.0191,0.232,-1.7799,-0.863,-0.1853,0.129,5.4218,0.1138,0.0453
EM_Net_T_Bill,0.2526,1.782,0.0528,0.3642,-2.3619,-0.8692,-0.2122,0.0737,4.2659,0.2091,0.087


nothing

"Correlations"

5×5 Matrix{Float64}:
  1.0        0.914307  -0.377797   0.96164    0.747361
  0.914307   1.0       -0.316058   0.782936   0.627688
 -0.377797  -0.316058   1.0       -0.357511  -0.405884
  0.96164    0.782936  -0.357511   1.0        0.716905
  0.747361   0.627688  -0.405884   0.716905   1.0

In [7]:
display(returns_summarystats(to(from(merge(dy, term), start),end_d), freq))


,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Dividend_Yeld,0.5539,-12.9938,-15.7151,-15.2616,-17.9466,0.3112,-16.38,0.9467,-0.3543,-28.373,-15.82
Yeld_10Y_T_Bill,0.0059,0.038,0.0158,0.025,-0.0136,-0.185,0.0068,0.884,-0.6223,2.6726,0.0158


nothing

In [25]:
assets_names =  [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
data = real_returns[assets_names]
T, K = size(data)

model = mvnormal_lkjcholesky_model(values(data), K, T)

# Sample from the posterior
chain = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(chain))
display(quantile(chain))


Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0015625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.003125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:00:35
Sampling (4 threads)  50%|████████████████              |  ETA: 0:00:13
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:00:06
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:17
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:17


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        μ[1]    0.0176    0.0061    0.0001   3291.8524   3187.0415    0.9999   ⋯
        μ[2]    0.0059    0.0037    0.0001   4984.4168   3467.5333    1.0016   ⋯
        μ[3]    0.0062    0.0070    0.0001   2993.6409   2970.1050    1.0006   ⋯
        μ[4]    0.0145    0.0104    0.0002   3048.5318   2864.4937    1.0007   ⋯
        σ[1]    0.0721    0.0043    0.0001   2350.2380   2563.0992    1.0000   ⋯
        σ[2]    0.0449    0.0028    0.0000   4245.1198   2956.5993    0.9998   ⋯
        σ[3]    0.0825    0.0049    0.0001   2384.3682   2726.8641    1.0006   ⋯
        σ[4]    0.1240    0.0074    0.0002   2264.7775   2698.7037    0.9998   ⋯
   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]   -0.3035    0.0744    0.0013   3434.1173   2979.0842    1.0007   ⋯
   L.L[3

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

        μ[1]    0.0054    0.0135    0.0177    0.0217    0.0294
        μ[2]   -0.0014    0.0035    0.0060    0.0084    0.0132
        μ[3]   -0.0077    0.0016    0.0062    0.0108    0.0198
        μ[4]   -0.0059    0.0075    0.0144    0.0216    0.0350
        σ[1]    0.0644    0.0691    0.0718    0.0748    0.0812
        σ[2]    0.0398    0.0430    0.0448    0.0467    0.0506
        σ[3]    0.0737    0.0791    0.0822    0.0856    0.0927
        σ[4]    0.1107    0.1188    0.1237    0.1288    0.1400
   L.L[1, 1]    1.0000    1.0000    1.0000    1.0000    1.0000
   L.L[2, 1]   -0.4436   -0.3559   -0.3055   -0.2534   -0.1535
   L.L[3, 1]    0.6874    0.7394    0.7640    0.7863    0.8244
   L.L[4, 1]    0.4908    0.5645    0.5993    0.6343    0.6951
   L.L[2, 2]    0.8962    0.9345    0.9522    0.9674    0.9882
   L.L[3, 2]   -0.2180   -0.1509   -0.1150

In [8]:
μ = mean(group(chain, :μ)).nt.mean
σ = mean(group(chain, :σ)).nt.mean
L = mean(group(chain, "L.L")).nt.mean

Σ = est_covariance(σ, L)

n_samples = 10000
n_steps = 25 * freq
scenarios = zeros(K, n_steps, n_samples)

for t in 1:n_steps
    scenarios[:,t,:]= rand(MvNormal(μ, Σ), 10000)
end

#= for k in 1:K
    display(round.(quantile(scenarios[k,1,:], [.025, .25, .5, .75, .975]), digits=2))
end =#

ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years, assets_names)

for a in 1:length(assets_names)
    print_scenarios_percentiles(ret_in_years[a,:,:], [.03, .25, .5, .75, .97], string(assets_names[a]))
end  

,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.0721,0.0232,0.0282,0.06
5,0.071,0.0237,0.0258,0.0578
10,0.0713,0.0235,0.0263,0.0588
25,0.0706,0.0234,0.0255,0.058


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.1438,0.0884,0.165,0.2458
5,0.0641,0.0402,0.0739,0.111
10,0.0372,0.0233,0.0428,0.0639
25,0.0291,0.0181,0.0332,0.0493


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,-0.0089,0.0182,-0.004,0.0266
5,0.0367,0.0278,-0.0033,-0.0344
10,-0.0005,0.0059,-0.0097,0.0008
25,-0.0087,-0.0203,-0.0076,0.005


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.0228,0.0715,0.0354,-0.0256
5,-0.0649,-0.0339,-0.0843,-0.035
10,0.02,-0.0603,-0.1333,-0.0007
25,-0.0915,-0.0281,-0.113,-0.0184


,0.03,0.25,0.5,0.75,0.97
1,-0.1953,-0.0247,0.0727,0.1686,0.3381
2,-0.0477,0.0275,0.0705,0.1154,0.1927
3,0.0008,0.0466,0.0714,0.0963,0.1418
4,0.0165,0.0506,0.0707,0.0906,0.1248


,0.03,0.25,0.5,0.75,0.97
1,-0.1426,-0.036,0.0232,0.0829,0.1886
2,-0.0511,-0.0039,0.0238,0.0511,0.0987
3,-0.0203,0.0077,0.0235,0.0394,0.0672
4,-0.0108,0.0112,0.0237,0.0358,0.0575


,0.03,0.25,0.5,0.75,0.97
1,-0.2823,-0.0819,0.029,0.14,0.3412
2,-0.1137,-0.0249,0.0264,0.0762,0.164
3,-0.0549,-0.003,0.027,0.0553,0.1047
4,-0.0373,0.0031,0.0253,0.0482,0.0877


,0.03,0.25,0.5,0.75,0.97
1,-0.4011,-0.1055,0.0594,0.2264,0.5242
2,-0.1511,-0.0166,0.0589,0.134,0.2607
3,-0.0624,0.0151,0.0594,0.1024,0.178
4,-0.0348,0.0248,0.0578,0.0911,0.1507


In [9]:
n_posterior_samples = 1000
n_samples = 10
n_steps = 100
scenarios = zeros(K, n_steps, n_posterior_samples)


for s in 1:n_posterior_samples
    
    μ = collect(Iterators.flatten(mean(group(chain, :μ)[s,:,:].value.data, dims=3)))
    σ = collect(Iterators.flatten(mean(group(chain, :σ)[s,:,:].value.data, dims=3)))
    L = collect(Iterators.flatten(mean(group(chain, "L.L")[s,:,:].value.data, dims=3)))
    Σ = est_covariance(σ, L)

    for t in 1:n_steps
    scenarios[:,t,s]= rand(MvNormal(μ, Σ), 1)
    end

end

#= for k in 1:K
    display(round.(quantile(scenarios[k,1,:], [.025, .25, .5, .75, .975]), digits=2))
end =#
ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years, assets_names)

,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.0738,0.0191,0.0303,0.0582
5,0.0746,0.0231,0.0306,0.0641
10,0.0708,0.0229,0.026,0.0589
25,0.0704,0.0235,0.0255,0.0579


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.146,0.0904,0.1617,0.2367
5,0.0675,0.0422,0.0736,0.1145
10,0.0405,0.0249,0.0455,0.0688
25,0.0311,0.0199,0.0351,0.054


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.0763,-0.0613,0.0192,-0.0309
5,0.0007,0.0622,0.0002,0.0096
10,0.0217,-0.1205,0.129,-0.0287
25,0.1284,-0.2391,0.1088,0.193


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,-0.3891,0.1998,-0.0123,-0.2354
5,0.0562,0.1728,0.0863,0.1005
10,0.0921,0.1252,0.1819,0.069
25,0.2196,0.3024,0.1668,0.2513


In [10]:
assets_names =  [:USA_Net_CPI, :Price_10Y_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
data = real_returns[assets_names]
T, K = size(data)

Y, X = prepare_var_data(values(data), 1, Matrix{Float64}(undef, 0, 0), true)

model = var_model(Y, X, 1)

# Sample from the posterior
chain_var = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(chain_var))
display(quantile(chain_var))

Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.00078125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0015625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.00078125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:02:02
Sampling (4 threads)  50%|████████████████              |  ETA: 0:01:42
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:00:36
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:01:49
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:01:49


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

      μ_β[1]    0.0160    0.0064    0.0001   2956.4879   3251.5742    1.0005   ⋯
      μ_β[2]    0.0045    0.0041    0.0001   4220.4094   3438.5824    0.9999   ⋯
      μ_β[3]    0.0052    0.0074    0.0001   2637.9563   3047.9834    1.0013   ⋯
      μ_β[4]    0.0135    0.0112    0.0002   2887.8748   2965.0832    1.0008   ⋯
      μ_β[5]    0.1023    0.1349    0.0031   1912.1658   2674.0579    1.0000   ⋯
      μ_β[6]    0.1134    0.0834    0.0017   2416.2092   2797.8932    1.0002   ⋯
      μ_β[7]    0.0431    0.1526    0.0036   1758.6053   2317.0684    1.0010   ⋯
      μ_β[8]   -0.1656    0.2300    0.0054   1795.9293   2279.3780    1.0013   ⋯
      μ_β[9]    0.0412    0.1506    0.0030   2545.8998   3090.7251    1.0000   ⋯
     μ_β[10]    0.0256    0.0968    0.0016   3721.8051   2962.1318    1.0013   ⋯
     μ_β

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

      μ_β[1]    0.0035    0.0118    0.0159    0.0204    0.0283
      μ_β[2]   -0.0037    0.0018    0.0045    0.0072    0.0125
      μ_β[3]   -0.0093    0.0002    0.0054    0.0102    0.0194
      μ_β[4]   -0.0083    0.0057    0.0135    0.0211    0.0358
      μ_β[5]   -0.1580    0.0114    0.1025    0.1918    0.3666
      μ_β[6]   -0.0494    0.0573    0.1131    0.1721    0.2692
      μ_β[7]   -0.2561   -0.0576    0.0434    0.1461    0.3365
      μ_β[8]   -0.6271   -0.3142   -0.1665   -0.0104    0.2832
      μ_β[9]   -0.2475   -0.0632    0.0416    0.1427    0.3381
     μ_β[10]   -0.1646   -0.0387    0.0257    0.0899    0.2176
     μ_β[11]   -0.4087   -0.1900   -0.0769    0.0400    0.2614
     μ_β[12]   -0.3601   -0.0161    0.1626    0.3341    0.6773
     μ_β[13]   -0.2228   -0.0483    0.0366    0.1290    0.2920
     μ_β[14]   -0.2236   -0.1214   -0.0641

In [11]:
n_assets = length(assets_names)
p = 1
betas = reshape(mean(group(chain_var, :μ_β)).nt.mean, n_assets, n_assets * p + 1)
sigmas = mean(group(chain_var, :σ)).nt.mean
L = mean(group(chain_var, "L.L")).nt.mean
Σ = est_covariance(sigmas, L)

initial_returns = values(data)[end, :]

K= n_assets
n_samples = 10_000
n_steps = 25 * freq
scenarios = zeros(K, n_steps, n_samples)

predictors0 = vcat(one(Float64), initial_returns)

scenarios[:,1,:] = rand(MvNormal(betas*predictors0, Σ), n_samples)

for s in 1:n_samples
    for t in 2:n_steps
        predictors = vcat(one(Float64), scenarios[:,t-1,s])
        scenarios[:,t,s] = rand(MvNormal(betas*predictors, Σ))
    end
end

ret_in_years = cum_returns_in_periods(scenarios, [1,5, 15, 25], freq, true)
print_scenarios_summary(ret_in_years, assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string(assets_names[a]))
end 

,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.0755,0.0225,0.0304,0.0516
5,0.0729,0.0244,0.0251,0.0528
10,0.0724,0.0244,0.0249,0.0535
25,0.0728,0.0244,0.025,0.0538


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,0.1573,0.0934,0.1776,0.2584
5,0.0727,0.0417,0.0805,0.1168
10,0.0429,0.024,0.0475,0.068
25,0.0333,0.0188,0.0368,0.0532


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,-0.0412,0.0037,-0.0366,-0.0432
5,0.0048,0.0057,-0.0085,-0.0648
10,0.0174,-0.0226,-0.01,-0.0302
25,0.0064,-0.0593,0.002,-0.0384


,USA_Net_CPI,Price_10Y_CPI,World_ex_USA_CPI,EM_Net_CPI
1,-0.021,-0.0105,-0.0127,0.0682
5,-0.014,0.0682,-0.0352,-0.0259
10,-0.0043,-0.0384,0.0089,0.0242
25,0.0471,-0.1175,0.0316,0.099


,0.03,0.25,0.5,0.75,0.97
1,-0.2233,-0.0315,0.0773,0.1831,0.3653
2,-0.0627,0.0239,0.073,0.1221,0.2092
3,-0.0079,0.0436,0.0725,0.1008,0.1537
4,0.0099,0.0504,0.0728,0.0952,0.136


,0.03,0.25,0.5,0.75,0.97
1,-0.1502,-0.0406,0.0226,0.0871,0.1982
2,-0.0541,-0.003,0.0239,0.0525,0.1028
3,-0.0214,0.0084,0.0242,0.0408,0.0691
4,-0.0107,0.0116,0.0246,0.0374,0.0589


,0.03,0.25,0.5,0.75,0.97
1,-0.3095,-0.0886,0.0324,0.1502,0.3606
2,-0.1262,-0.0296,0.0252,0.0803,0.1751
3,-0.065,-0.0068,0.0244,0.0571,0.1139
4,-0.0448,0.0007,0.0249,0.0497,0.0942


,0.03,0.25,0.5,0.75,0.97
1,-0.4343,-0.121,0.0514,0.228,0.5362
2,-0.1683,-0.0259,0.0544,0.132,0.2676
3,-0.0751,0.0074,0.0547,0.0992,0.1827
4,-0.0474,0.0191,0.0541,0.0889,0.1526


In [12]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :Price_10Y_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill), start),end_d)
assets_names = colnames(data)

T, K = size(data)

Y, X = prepare_var_data(values(data), 1, Matrix{Float64}(undef, 0, 0), true)

model = var_model(Y, X, 1)

# Sample from the posterior
chain_var = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(group(chain_var, :μ_β)))
display(summarystats(group(chain_var, "L.L")))
display(summarystats(group(chain_var, :σ)))
display(quantile(chain_var))


Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.00078125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0015625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 9.765625e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:04:42
Sampling (4 threads)  50%|████████████████              |  ETA: 0:05:09
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:01:49
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:05:32
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:05:32


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

      μ_β[1]    0.0136    0.0068    0.0001   3677.1767   3303.2712    1.0019   ⋯
      μ_β[2]    0.0060    0.0037    0.0001   4435.9962   3762.3042    0.9998   ⋯
      μ_β[3]    0.0034    0.0078    0.0001   3372.9016   3250.8835    1.0017   ⋯
      μ_β[4]    0.0132    0.0119    0.0002   3439.6495   3160.6277    1.0021   ⋯
      μ_β[5]    0.0003    0.0008    0.0000   4258.3982   3293.8450    0.9999   ⋯
      μ_β[6]    0.1464    0.1446    0.0032   2051.7040   2469.7685    1.0034   ⋯
      μ_β[7]    0.0798    0.0831    0.0015   3250.1380   3173.9968    1.0011   ⋯
      μ_β[8]    0.0259    0.1686    0.0037   2063.9644   2612.5747    1.0038   ⋯
      μ_β[9]   -0.1948    0.2541    0.0055   2154.9477   2555.9444    1.0032   ⋯
     μ_β[10]   -0.0023    0.0169    0.0002   4862.0416   3065.6215    1.0002   ⋯
     μ_β

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]   -0.3452    0.0748    0.0011   5117.9505   2498.6661    1.0007   ⋯
   L.L[3, 1]    0.7752    0.0345    0.0006   3269.8847   3082.3955    0.9995   ⋯
   L.L[4, 1]    0.6271    0.0520    0.0009   3472.5702   3028.0122    1.0001   ⋯
   L.L[5, 1]   -0.0661    0.0844    0.0011   5792.6171   3143.2462    1.0004   ⋯
   L.L[2, 2]    0.9351    0.0275    0.0004   5117.9505   2498.6661    1.0008   ⋯
   L.L[3, 2]   -0.0767    0.0544    0.0007   5440.4754   3292.9361    1.0005   ⋯
   L.L[4, 2]   -0.1872    0.0657    0.0009   5370.8670   3364.5023    1.0002   ⋯
   L.L[5, 2]    0.2329    0.0797    0.0011   5658.0503   3250.3192    1.0004   ⋯
   L.L[3, 3]    0.6223    0.0415    0.0007   3289.4070   3094.5722    0.9995   ⋯
   L.L[4

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        σ[1]    0.0734    0.0044    0.0001   3339.1394   3216.6758    1.0000   ⋯
        σ[2]    0.0421    0.0026    0.0000   5960.9511   3299.0244    1.0003   ⋯
        σ[3]    0.0846    0.0051    0.0001   3102.6160   3439.2818    1.0003   ⋯
        σ[4]    0.1285    0.0078    0.0001   3470.9318   2894.5844    0.9999   ⋯
        σ[5]    0.0085    0.0005    0.0000   6508.9491   3274.0075    1.0012   ⋯
                                                                1 column omitted


Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

      μ_β[1]    0.0004    0.0090    0.0134    0.0183    0.0269
      μ_β[2]   -0.0014    0.0034    0.0059    0.0086    0.0134
      μ_β[3]   -0.0115   -0.0019    0.0034    0.0086    0.0193
      μ_β[4]   -0.0102    0.0052    0.0132    0.0212    0.0373
      μ_β[5]   -0.0012   -0.0003    0.0003    0.0008    0.0018
      μ_β[6]   -0.1416    0.0480    0.1438    0.2443    0.4274
      μ_β[7]   -0.0872    0.0241    0.0816    0.1353    0.2412
      μ_β[8]   -0.3124   -0.0847    0.0242    0.1405    0.3581
      μ_β[9]   -0.7015   -0.3599   -0.1943   -0.0289    0.3093
     μ_β[10]   -0.0340   -0.0137   -0.0026    0.0092    0.0312
     μ_β[11]   -0.1938    0.0313    0.1469    0.2610    0.4771
     μ_β[12]   -0.2474   -0.1187   -0.0543    0.0130    0.1393
     μ_β[13]   -0.3912   -0.1351   -0.0009    0.1349    0.3788
     μ_β[14]   -0.4277   -0.0306    0.1736

In [38]:
n_assets = K
p =1
betas = reshape(mean(group(chain_var, :μ_β)).nt.mean, n_assets, n_assets * p + 1)
sigmas = mean(group(chain_var, :σ)).nt.mean
L = mean(group(chain_var, "L.L")).nt.mean
Σ = est_covariance(sigmas, L)

initial_returns = values(data)[end, :]

n_samples = 10_000
n_steps = 25 * freq
scenarios = zeros(n_assets, n_steps, n_samples)

predictors0 = vcat(one(Float64), initial_returns)

scenarios[:,1,:] = rand(MvNormal(betas*predictors0, Σ), n_samples)

for s in 1:n_samples
    for t in 2:n_steps
        predictors = vcat(one(Float64), scenarios[:,t-1,s])
        scenarios[:,t,s] = rand(MvNormal(betas*predictors, Σ))
    end
end

periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(assets_names[a]))
end  

,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI
1,0.0573,0.0328,0.0073,0.0626,0.0033
5,0.0642,0.0273,0.0144,0.0512,0.0009
15,0.0664,0.0267,0.0165,0.0514,0.0004
25,0.0675,0.0263,0.0179,0.0523,0.0003


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI
1,0.1638,0.0845,0.187,0.2728,0.022
5,0.0759,0.0379,0.0869,0.1234,0.0107
15,0.0446,0.022,0.0509,0.0715,0.0063
25,0.0344,0.0169,0.0395,0.0556,0.0049


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI
1,-0.0221,-0.0047,0.0115,-0.0364,-0.0368
5,-0.0336,0.0098,-0.0282,-0.0302,-0.0147
15,-0.0309,-0.0049,0.0036,0.0245,-0.0335
25,-0.0121,0.0109,0.0008,-0.006,0.0003


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI
1,-0.157,-0.0214,-0.0354,-0.0465,-0.0052
5,0.0544,-0.0259,0.0045,0.0496,0.0819
15,-0.026,0.0001,-0.0149,-0.0203,-0.0042
25,-0.0342,0.0075,-0.0314,-0.0729,0.0296


,0.03,0.25,0.5,0.75,0.97
1,-0.2511,-0.0547,0.0586,0.1696,0.3594
2,-0.0807,0.0145,0.064,0.1156,0.2058
3,-0.018,0.0365,0.0667,0.0969,0.149
4,0.0025,0.0441,0.0674,0.0912,0.132


,0.03,0.25,0.5,0.75,0.97
1,-0.1273,-0.0236,0.0328,0.0907,0.1894
2,-0.0439,0.0015,0.027,0.0533,0.0996
3,-0.014,0.0121,0.0265,0.0417,0.0674
4,-0.0057,0.0148,0.0263,0.0376,0.0583


,0.03,0.25,0.5,0.75,0.97
1,-0.3405,-0.1204,0.0077,0.1313,0.3618
2,-0.1512,-0.0431,0.0138,0.0737,0.1755
3,-0.0788,-0.0175,0.0164,0.0508,0.1125
4,-0.0557,-0.009,0.0177,0.0444,0.0925


,0.03,0.25,0.5,0.75,0.97
1,-0.4517,-0.1233,0.0656,0.2488,0.5632
2,-0.1812,-0.031,0.0506,0.1351,0.2828
3,-0.0818,0.0028,0.0511,0.0997,0.1888
4,-0.0525,0.014,0.0521,0.0903,0.1574


,0.03,0.25,0.5,0.75,0.97
1,-0.0386,-0.0115,0.0037,0.0182,0.0447
2,-0.0193,-0.0063,0.0009,0.0081,0.0213
3,-0.0116,-0.0039,0.0004,0.0047,0.0121
4,-0.0089,-0.003,0.0003,0.0036,0.0092


### excess returm s with log div yeld

In [9]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :Price_10Y_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, dy), start),end_d)
assets_names = colnames(data)

T, K = size(data)

Y, X = prepare_var_data(values(data), 1, Matrix{Float64}(undef, 0, 0), true)

([0.09021979330840788 0.07990917017417429 … 0.009082094488073227 -3.460790033882712; 0.07733294464695713 -0.013297473493224011 … 0.009060812019922284 -3.4712363219239966; … ; 0.06701658746523402 -0.0662456632762702 … 0.006502044752743819 -4.115841986801027; 0.004614046576888381 0.05456218837657381 … 0.010346584402051042 -4.224326679669056], [1.0 0.034592591476941284 … 0.004750856136060879 -3.400202177406109; 1.0 0.09021979330840788 … 0.009082094488073227 -3.460790033882712; … ; 1.0 0.039902835640925206 … 0.004933523891324452 -4.180809147785739; 1.0 0.06701658746523402 … 0.006502044752743819 -4.115841986801027])

In [10]:
model = var_model(Y, X, 1)

# Sample from the posterior
chain_var = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(group(chain_var, :μ_β)))
display(summarystats(group(chain_var, "L.L")))
display(summarystats(group(chain_var, :σ)))
display(quantile(chain_var))

Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 1.220703125e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 9.765625e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:48:00
Sampling (4 threads)  50%|████████████████              |  ETA: 0:17:08
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:06:40
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:57:00
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:57:00


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

      μ_β[1]    0.0880    0.0903    0.0024   1356.4433   1990.6722    1.0072   ⋯
      μ_β[2]    0.0660    0.0542    0.0012   1955.1823   2447.2190    1.0012   ⋯
      μ_β[3]   -0.0484    0.1065    0.0029   1370.3367   2036.2504    1.0077   ⋯
      μ_β[4]    0.1867    0.1608    0.0041   1520.0318   2282.6786    1.0063   ⋯
      μ_β[5]    0.0185    0.0110    0.0002   3732.7126   2946.1043    1.0007   ⋯
      μ_β[6]   -0.1677    0.0962    0.0023   1760.9960   2430.3142    1.0026   ⋯
      μ_β[7]    0.1456    0.1449    0.0033   1908.2568   1916.5132    1.0011   ⋯
      μ_β[8]    0.0779    0.0831    0.0017   2445.6671   2454.2128    1.0007   ⋯
      μ_β[9]    0.0302    0.1659    0.0039   1851.0382   2226.7366    1.0014   ⋯
     μ_β[10]   -0.1953    0.2507    0.0054   2123.9666   2864.1866    1.0028   ⋯
     μ_β

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]   -0.3481    0.0761    0.0013   3695.3716   3082.3440    1.0001   ⋯
   L.L[3, 1]    0.7737    0.0354    0.0007   2349.8547   2994.3731    1.0010   ⋯
   L.L[4, 1]    0.6153    0.0551    0.0011   2338.9914   2745.7586    1.0000   ⋯
   L.L[5, 1]   -0.0728    0.0835    0.0013   4171.1075   2902.5588    1.0018   ⋯
   L.L[6, 1]   -0.5610    0.0589    0.0010   3263.7062   2924.5007    1.0004   ⋯
   L.L[2, 2]    0.9339    0.0283    0.0005   3695.3716   3082.3440    1.0001   ⋯
   L.L[3, 2]   -0.0660    0.0563    0.0010   3424.3301   3096.4822    0.9998   ⋯
   L.L[4, 2]   -0.1989    0.0665    0.0012   2997.6807   3084.3129    1.0011   ⋯
   L.L[5, 2]    0.2164    0.0816    0.0015   2861.3907   2720.4502    1.0018   ⋯
   L.L[6

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        σ[1]    0.0727    0.0044    0.0001   2469.0614   2973.2839    1.0000   ⋯
        σ[2]    0.0422    0.0027    0.0000   3515.7971   2734.2758    0.9997   ⋯
        σ[3]    0.0842    0.0052    0.0001   2364.4001   2563.4369    1.0000   ⋯
        σ[4]    0.1276    0.0077    0.0001   2743.9089   3003.2416    1.0005   ⋯
        σ[5]    0.0085    0.0005    0.0000   3078.3402   2882.3104    1.0000   ⋯
        σ[6]    0.0749    0.0046    0.0001   3582.7819   3253.8599    0.9996   ⋯
                                                                1 column omitted


Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

      μ_β[1]   -0.0870    0.0255    0.0888    0.1483    0.2622
      μ_β[2]   -0.0404    0.0302    0.0655    0.1034    0.1700
      μ_β[3]   -0.2600   -0.1193   -0.0493    0.0222    0.1609
      μ_β[4]   -0.1320    0.0773    0.1883    0.2890    0.5040
      μ_β[5]   -0.0032    0.0110    0.0186    0.0257    0.0396
      μ_β[6]   -0.3536   -0.2317   -0.1672   -0.1051    0.0248
      μ_β[7]   -0.1381    0.0469    0.1448    0.2402    0.4365
      μ_β[8]   -0.0854    0.0222    0.0790    0.1338    0.2481
      μ_β[9]   -0.2945   -0.0798    0.0272    0.1407    0.3538
     μ_β[10]   -0.6661   -0.3676   -0.2008   -0.0286    0.2945
     μ_β[11]   -0.0360   -0.0138   -0.0029    0.0078    0.0283
     μ_β[12]   -0.2941   -0.0965    0.0038    0.1046    0.3031
     μ_β[13]   -0.2008    0.0147    0.1347    0.2420    0.4712
     μ_β[14]   -0.2596   -0.1301   -0.0634

In [11]:
n_assets = K
p =1
betas = reshape(mean(group(chain_var, :μ_β)).nt.mean, n_assets, n_assets * p + 1)
sigmas = mean(group(chain_var, :σ)).nt.mean
L = mean(group(chain_var, "L.L")).nt.mean
Σ = est_covariance(sigmas, L)

initial_returns = values(data)[end, :]

n_samples = 10_000
n_steps = 25 * freq
scenarios = zeros(n_assets, n_steps, n_samples)

predictors0 = vcat(one(Float64), initial_returns)

scenarios[:,1,:] = rand(MvNormal(betas*predictors0, Σ), n_samples)

for s in 1:n_samples
    for t in 2:n_steps
        predictors = vcat(one(Float64), scenarios[:,t-1,s])
        scenarios[:,t,s] = rand(MvNormal(betas*predictors, Σ))
    end
end

periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(assets_names[a]))
end  

,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.0343,0.0104,0.0232,0.0068,-0.0032,-16.9071
5,0.049,0.0099,0.0305,0.0123,-0.0047,-16.7024
15,0.0542,0.0132,0.028,0.0207,-0.0038,-16.4572
25,0.0564,0.0147,0.0272,0.0246,-0.0034,-16.3689


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.1641,0.0832,0.1881,0.2682,0.0217,0.3914
5,0.0728,0.0386,0.0895,0.1215,0.0105,0.5991
15,0.0403,0.0242,0.0537,0.0702,0.0065,0.6149
25,0.0304,0.0194,0.0421,0.0548,0.0052,0.5495


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,-0.0356,0.018,-0.0098,0.0272,0.0046,-0.0393
5,0.0136,0.0226,-0.0098,-0.0159,-0.041,-0.0179
15,0.0198,-0.0268,0.0335,0.0437,-0.0472,-0.003
25,0.0007,-0.0307,-0.0212,0.0047,-0.0303,0.0125


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.0067,0.0179,-0.0348,-0.035,-0.0391,0.0107
5,0.0005,0.03,0.0441,-0.0241,0.1133,-0.0192
15,-0.0496,-0.0106,-0.0496,-0.0731,-0.0636,0.0534
25,-0.0359,-0.0315,-0.0696,0.0184,-0.0984,-0.0073


,0.03,0.25,0.5,0.75,0.97
1,-0.2734,-0.0764,0.0353,0.147,0.3381
2,-0.0868,-0.0008,0.0491,0.0983,0.1869
3,-0.0213,0.0268,0.0538,0.0816,0.1292
4,0.0004,0.0358,0.0564,0.0769,0.1134


,0.03,0.25,0.5,0.75,0.97
1,-0.1463,-0.0459,0.0101,0.0665,0.1659
2,-0.0622,-0.0162,0.01,0.0356,0.0827
3,-0.0327,-0.0032,0.0135,0.0293,0.0589
4,-0.0215,0.0015,0.0147,0.028,0.0507


,0.03,0.25,0.5,0.75,0.97
1,-0.3325,-0.1039,0.0221,0.1507,0.3692
2,-0.1401,-0.0305,0.0322,0.0908,0.1973
3,-0.0716,-0.009,0.0282,0.0647,0.1282
4,-0.0527,-0.0015,0.0277,0.0554,0.105


,0.03,0.25,0.5,0.75,0.97
1,-0.4863,-0.1771,0.0056,0.1871,0.5144
2,-0.219,-0.0686,0.0131,0.0947,0.2364
3,-0.1091,-0.0276,0.0198,0.0678,0.1541
4,-0.0779,-0.012,0.0247,0.0615,0.129


,0.03,0.25,0.5,0.75,0.97
1,-0.0442,-0.0179,-0.0031,0.0114,0.0373
2,-0.0246,-0.0118,-0.0047,0.0024,0.0151
3,-0.0162,-0.0082,-0.0039,0.0006,0.0082
4,-0.0131,-0.007,-0.0035,0.0002,0.0065


,0.03,0.25,0.5,0.75,0.97
1,-17.656,-17.1711,-16.9045,-16.6399,-16.1854
2,-17.8367,-17.1097,-16.6995,-16.2932,-15.5919
3,-17.6071,-16.8719,-16.469,-16.044,-15.3043
4,-17.4105,-16.745,-16.3729,-15.9987,-15.3316


In [21]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, dy, term), start),end_d)
assets_names = colnames(data)
T, K = size(data)

Y, X = prepare_var_data(values(data), 1, Matrix{Float64}(undef, 0, 0), true)


([0.09021979330840788 0.009082094488073227 -3.460790033882712 -0.00046242775390518964; 0.07733294464695713 0.009060812019922284 -3.4712363219239966 0.005183756863788608; … ; 0.06701658746523402 0.006502044752743819 -4.115841986801027 -0.006955389861931838; 0.004614046576888381 0.010346584402051042 -4.224326679669056 -0.01300706937754429], [1.0 0.034592591476941284 … -3.400202177406109 0.004401254124713594; 1.0 0.09021979330840788 … -3.460790033882712 -0.00046242775390518964; … ; 1.0 0.039902835640925206 … -4.180809147785739 -0.012920694656526427; 1.0 0.06701658746523402 … -4.115841986801027 -0.006955389861931838])

In [22]:
model = var_model(Y, X, 1)

# Sample from the posterior
chain_var = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(group(chain_var, :μ_β)))
display(summarystats(group(chain_var, "L.L")))
display(summarystats(group(chain_var, :σ)))
display(quantile(chain_var))

Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 4.8828125e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 1.220703125e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 0:13:14
Sampling (4 threads)  50%|████████████████              |  ETA: 0:12:22
Sampling (4 threads)  75%|███████████████████████       |  ETA: 0:06:10
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:24:27
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:24:27


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

      μ_β[1]    0.2059    0.1269    0.0303     19.3909     49.9755    1.3174   ⋯
      μ_β[2]    0.0256    0.0113    0.0002   2760.8512   2708.9180    1.0039   ⋯
      μ_β[3]   -0.2349    0.1108    0.0169     44.1977    121.1424    1.1156   ⋯
      μ_β[4]    0.0098    0.0069    0.0003    585.5387   2750.7119    1.0153   ⋯
      μ_β[5]    0.1195    0.0874    0.0021   1826.0863   2100.0072    1.0025   ⋯
      μ_β[6]    0.0049    0.0099    0.0002   4090.1961   2957.0925    1.0008   ⋯
      μ_β[7]   -0.0296    0.0873    0.0019   2186.6104   2788.2151    1.0004   ⋯
      μ_β[8]   -0.0044    0.0058    0.0001   3128.4713   2738.3392    1.0005   ⋯
      μ_β[9]   -2.0046    1.0386    0.2812     15.2879     48.4827    1.4594   ⋯
     μ_β[10]    0.2401    0.0904    0.0018   2509.0812   2919.2684    1.0048   ⋯
     μ_β

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]   -0.1288    0.0836    0.0015   3210.3012   3015.8178    1.0005   ⋯
   L.L[3, 1]   -0.5760    0.0577    0.0013   1916.0076   2444.6392    1.0102   ⋯
   L.L[4, 1]    0.2066    0.0812    0.0015   2761.5054   2575.6672    1.0022   ⋯
   L.L[2, 2]    0.9881    0.0120    0.0002   2955.7249   3015.8178    1.0004   ⋯
   L.L[3, 2]    0.0001    0.0692    0.0011   3738.8942   2857.1574    1.0005   ⋯
   L.L[4, 2]   -0.1685    0.0808    0.0014   3403.1407   2659.4382    1.0014   ⋯
   L.L[3, 3]    0.8115    0.0405    0.0009   1918.9249   2492.9880    1.0099   ⋯
   L.L[4, 3]    0.0602    0.0849    0.0045    353.0887    889.5907    1.0176   ⋯
   L.L[4, 4]    0.9510    0.0233    0.0005   2188.6781   2884.7290    1.0029   ⋯
        

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        σ[1]    0.0742    0.0049    0.0007     57.3750    161.6925    1.0868   ⋯
        σ[2]    0.0083    0.0005    0.0000   3928.1317   2781.6207    1.0014   ⋯
        σ[3]    0.0748    0.0047    0.0001   2595.5607   2948.4095    1.0079   ⋯
        σ[4]    0.0049    0.0003    0.0000   3241.0404   2811.6527    1.0005   ⋯
                                                                1 column omitted


Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

      μ_β[1]   -0.0164    0.1169    0.1919    0.2868    0.4803
      μ_β[2]    0.0039    0.0181    0.0254    0.0333    0.0478
      μ_β[3]   -0.4678   -0.3054   -0.2338   -0.1591   -0.0296
      μ_β[4]   -0.0041    0.0052    0.0099    0.0145    0.0233
      μ_β[5]   -0.0528    0.0607    0.1214    0.1795    0.2875
      μ_β[6]   -0.0148   -0.0017    0.0048    0.0116    0.0244
      μ_β[7]   -0.2013   -0.0893   -0.0306    0.0302    0.1402
      μ_β[8]   -0.0159   -0.0083   -0.0043   -0.0005    0.0072
      μ_β[9]   -3.9427   -2.8148   -1.8994   -1.2230   -0.2047
     μ_β[10]    0.0569    0.1782    0.2430    0.3025    0.4106
     μ_β[11]   -2.2458   -1.2341   -0.6334   -0.0311    1.0723
     μ_β[12]   -0.1413   -0.0717   -0.0347    0.0003    0.0729
     μ_β[13]   -0.0101    0.0223    0.0399    0.0602    0.1043
     μ_β[14]    0.0005    0.0041    0.0059

In [23]:
n_assets = K
p =1
betas = reshape(mean(group(chain_var, :μ_β)).nt.mean, n_assets, n_assets * p + 1)
sigmas = mean(group(chain_var, :σ)).nt.mean
L = mean(group(chain_var, "L.L")).nt.mean
Σ = est_covariance(sigmas, L)

initial_returns = values(data)[end, :]

n_samples = 10_000
n_steps = 25 * freq
scenarios = zeros(n_assets, n_steps, n_samples)

predictors0 = vcat(one(Float64), initial_returns)

scenarios[:,1,:] = rand(MvNormal(betas*predictors0, Σ), n_samples)

for s in 1:n_samples
    for t in 2:n_steps
        predictors = vcat(one(Float64), scenarios[:,t-1,s])
        scenarios[:,t,s] = rand(MvNormal(betas*predictors, Σ))
    end
end

periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(assets_names[a]))
end  

,USA_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.1435,0.014,-17.1106,-0.0365
5,0.1166,0.0039,-17.3059,-0.0094
15,0.0919,-0.0005,-17.1648,0.0124
25,0.0853,-0.0013,-17.0057,0.0209


,USA_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.1624,0.0207,0.3902,0.0239
5,0.0754,0.0118,0.6241,0.032
15,0.0465,0.0075,0.7778,0.0327
25,0.0375,0.0058,0.7726,0.0303


,USA_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,-0.0275,0.0178,0.0123,-0.0017
5,-0.024,-0.0074,0.0162,0.0332
15,-0.0285,0.0109,0.0425,0.0413
25,-0.0176,0.0012,0.0509,0.0442


,USA_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,-0.0096,-0.1369,0.009,-0.0332
5,-0.0319,-0.0445,0.0229,0.0354
15,-0.0163,0.057,-0.0204,-0.0218
25,-0.0552,0.0653,-0.0049,-0.0493


,0.03,0.25,0.5,0.75,0.97
1,-0.1646,0.0352,0.145,0.2524,0.446
2,-0.0255,0.0664,0.1162,0.168,0.2571
3,0.004,0.0607,0.0916,0.1238,0.1795
4,0.0146,0.0604,0.0852,0.1111,0.1555


,0.03,0.25,0.5,0.75,0.97
1,-0.0246,-0.0003,0.0138,0.0283,0.0528
2,-0.0183,-0.0041,0.004,0.012,0.0259
3,-0.0147,-0.0055,-0.0006,0.0046,0.0137
4,-0.0122,-0.0052,-0.0013,0.0026,0.0097


,0.03,0.25,0.5,0.75,0.97
1,-17.8414,-17.3739,-17.1129,-16.8455,-16.3771
2,-18.4903,-17.7211,-17.3082,-16.8894,-16.1257
3,-18.6091,-17.694,-17.1737,-16.6398,-15.6878
4,-18.4344,-17.5309,-17.0066,-16.4915,-15.5293


,0.03,0.25,0.5,0.75,0.97
1,-0.0817,-0.0528,-0.0365,-0.0202,0.0086
2,-0.0695,-0.031,-0.0098,0.0117,0.0513
3,-0.0484,-0.0094,0.0119,0.0344,0.0749
4,-0.0345,0.0,0.0205,0.0416,0.0781


In [28]:
start = Date(1989, 01, 01)
end_d = Date(2025, 01, 31)
assets_names = [:USA_Net_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
data = to(from(merge(real_returns[assets_names],  real_t_bill, real_10Y, cpi), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)

Y, X = prepare_var_data(values(data), 1, Matrix{Float64}(undef, 0, 0), true)
model = var_model(Y, X, 1)

# Sample from the posterior
chain_var = sample(model, NUTS(), MCMCThreads(), 1000, 4; discard_initial=1000)

# Summarize results
display(summarystats(group(chain_var, :μ_β)))
display(summarystats(group(chain_var, "L.L")))
display(summarystats(group(chain_var, :σ)))
display(quantile(chain_var))

Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 9.765625e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.0001953125
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 0.000390625
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
┌ Info: Found initial step size
│   ϵ = 4.8828125e-5
└ @ Turing.Inference C:\Users\matsz\.julia\packages\Turing\GSvwQ\src\mcmc\hmc.jl:213
Sampling (4 threads)  25%|████████                      |  ETA: 1:14:25
Sampling (4 threads)  50%|████████████████              |  ETA: 7:29:32
Sampling (4 threads)  75%|███████████████████████       |  ETA: 2:35:07
Sampling (4 threads) 100%|██████████████████████████████| Time: 7:49:02
Sampling (4 threads) 100%|██████████████████████████████| Time: 7:49:02


Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

      μ_β[1]    0.0178    0.0143    0.0002   3750.0840   3493.6410    0.9998   ⋯
      μ_β[2]    0.0175    0.0159    0.0003   3400.9542   3271.9361    1.0003   ⋯
      μ_β[3]    0.0088    0.0231    0.0004   3429.2544   3667.8587    1.0002   ⋯
      μ_β[4]   -0.0067    0.0018    0.0000   2112.0171   2632.0942    1.0005   ⋯
      μ_β[5]   -0.0062    0.0018    0.0000   2149.5990   2740.6865    1.0007   ⋯
      μ_β[6]    0.0068    0.0019    0.0000   2171.2666   2544.0179    1.0007   ⋯
      μ_β[7]    0.1200    0.1421    0.0024   3488.1350   3130.5519    1.0012   ⋯
      μ_β[8]    0.0251    0.1582    0.0028   3155.2960   2990.0541    1.0006   ⋯
      μ_β[9]   -0.2077    0.2446    0.0043   3287.1761   3196.7458    1.0017   ⋯
     μ_β[10]    0.0035    0.0141    0.0003   2686.7244   2978.0041    0.9998   ⋯
     μ_β

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

   L.L[1, 1]    1.0000    0.0000       NaN         NaN         NaN       NaN   ⋯
   L.L[2, 1]    0.7652    0.0359    0.0006   4155.8452   3393.0490    1.0011   ⋯
   L.L[3, 1]    0.6095    0.0533    0.0008   4192.5901   3070.8164    1.0001   ⋯
   L.L[4, 1]   -0.0165    0.0780    0.0012   4242.4817   2706.7620    1.0017   ⋯
   L.L[5, 1]    0.0166    0.0777    0.0012   4311.3744   2959.2855    1.0014   ⋯
   L.L[6, 1]    0.0343    0.0779    0.0012   4204.1397   2902.9851    1.0013   ⋯
   L.L[2, 2]    0.6414    0.0422    0.0007   4155.8452   3393.0490    1.0012   ⋯
   L.L[3, 2]    0.3554    0.0606    0.0008   5931.4270   2878.4382    1.0002   ⋯
   L.L[4, 2]   -0.2108    0.0785    0.0013   3522.4614   2469.3437    1.0042   ⋯
   L.L[5, 2]   -0.2163    0.0781    0.0013   3604.7752   2418.3235    1.0033   ⋯
   L.L[6

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        σ[1]    0.0725    0.0044    0.0001   4104.2939   3243.8771    1.0002   ⋯
        σ[2]    0.0821    0.0049    0.0001   4178.8876   3152.3654    1.0006   ⋯
        σ[3]    0.1260    0.0077    0.0001   4569.3703   3542.3345    1.0003   ⋯
        σ[4]    0.0073    0.0004    0.0000   2940.4930   2744.3290    1.0005   ⋯
        σ[5]    0.0072    0.0004    0.0000   2950.7340   2943.9778    1.0005   ⋯
        σ[6]    0.0075    0.0004    0.0000   2968.1483   2656.5415    1.0011   ⋯
                                                                1 column omitted


Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

      μ_β[1]   -0.0108    0.0082    0.0181    0.0276    0.0453
      μ_β[2]   -0.0134    0.0069    0.0172    0.0283    0.0485
      μ_β[3]   -0.0354   -0.0067    0.0086    0.0243    0.0544
      μ_β[4]   -0.0102   -0.0079   -0.0067   -0.0055   -0.0031
      μ_β[5]   -0.0097   -0.0074   -0.0062   -0.0050   -0.0027
      μ_β[6]    0.0032    0.0056    0.0068    0.0081    0.0104
      μ_β[7]   -0.1483    0.0204    0.1207    0.2155    0.4045
      μ_β[8]   -0.2896   -0.0790    0.0260    0.1341    0.3375
      μ_β[9]   -0.6924   -0.3722   -0.2096   -0.0471    0.2741
     μ_β[10]   -0.0244   -0.0061    0.0035    0.0130    0.0309
     μ_β[11]   -0.0256   -0.0075    0.0022    0.0114    0.0298
     μ_β[12]   -0.0329   -0.0147   -0.0050    0.0051    0.0239
     μ_β[13]   -0.2752   -0.0970   -0.0010    0.0939    0.2777
     μ_β[14]   -0.2779   -0.0785    0.0287

(142, 6)

In [37]:
p =1
n_assets = 6
betas = reshape(mean(group(chain_var, :μ_β)).nt.mean, n_assets, n_assets * p + 1)
sigmas = mean(group(chain_var, :σ)).nt.mean
L = mean(group(chain_var, "L.L")).nt.mean
Σ = est_covariance(sigmas, L)

initial_returns = values(data)[end, :]

n_samples = 10_000
n_steps = 25 * freq
scenarios = zeros(n_assets, n_steps, n_samples)

predictors0 = vcat(one(Float64), initial_returns)

scenarios[:,1,:] = rand(MvNormal(betas*predictors0, Σ), n_samples)

for s in 1:n_samples
    for t in 2:n_steps
        predictors = vcat(one(Float64), scenarios[:,t-1,s])
        scenarios[:,t,s] = rand(MvNormal(betas*predictors, Σ))
    end
end

periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.1015,0.0276,0.0544,0.0157,0.014,0.0323
5,0.0926,0.0244,0.051,0.011,0.0158,0.0303
15,0.0807,0.0243,0.0513,0.004,0.0145,0.0282
25,0.0776,0.026,0.0509,0.0008,0.0131,0.0274


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.1574,0.1793,0.2689,0.0147,0.0148,0.0155
5,0.0741,0.0805,0.1238,0.0088,0.0094,0.0074
15,0.0435,0.0459,0.0725,0.0085,0.0077,0.0048
25,0.0339,0.0352,0.0558,0.0077,0.0066,0.0039


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.0192,0.0274,-0.0013,-0.0102,-0.0139,0.0063
5,0.0091,0.0096,0.0336,-0.0389,0.0001,-0.0395
15,0.0125,-0.0116,0.0106,0.0171,0.0077,-0.0034
25,0.0199,-0.0072,0.0148,0.0193,0.0056,-0.0017


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,-0.0877,-0.0095,0.0396,0.0935,0.0227,0.0337
5,0.0293,0.0112,0.0471,0.0208,0.1205,-0.0094
15,-0.0073,-0.001,-0.0207,-0.1031,-0.0054,-0.0912
25,0.0796,-0.082,-0.0626,-0.0288,0.0044,0.0068


,0.03,0.25,0.5,0.75,0.97
1,-0.1942,-0.0051,0.1002,0.2076,0.4008
5,-0.0483,0.0419,0.0929,0.1422,0.2334
15,-0.0005,0.0519,0.0804,0.1103,0.162
25,0.0143,0.0548,0.077,0.1,0.1421


,0.03,0.25,0.5,0.75,0.97
1,-0.306,-0.0937,0.026,0.1468,0.3662
5,-0.1281,-0.0296,0.0246,0.0786,0.1747
15,-0.0617,-0.0064,0.0241,0.0555,0.1088
25,-0.0405,0.002,0.0256,0.0503,0.0919


,0.03,0.25,0.5,0.75,0.97
1,-0.4521,-0.1281,0.0559,0.2352,0.5618
5,-0.1813,-0.0321,0.0501,0.1337,0.2849
15,-0.0849,0.002,0.0503,0.1,0.1865
25,-0.0536,0.0129,0.0502,0.0887,0.1562


,0.03,0.25,0.5,0.75,0.97
1,-0.012,0.0058,0.0158,0.0255,0.0435
5,-0.0058,0.0051,0.011,0.0169,0.0272
15,-0.0117,-0.0017,0.004,0.0099,0.02
25,-0.0136,-0.0045,0.0007,0.006,0.0154


,0.03,0.25,0.5,0.75,0.97
1,-0.014,0.0041,0.014,0.0241,0.0419
5,-0.0018,0.0096,0.0158,0.022,0.0335
15,-0.0001,0.0094,0.0145,0.0197,0.0288
25,0.0009,0.0087,0.0131,0.0176,0.0255


,0.03,0.25,0.5,0.75,0.97
1,0.003,0.0219,0.0322,0.0429,0.0612
5,0.0163,0.0254,0.0304,0.0354,0.0442
15,0.0192,0.0249,0.0281,0.0314,0.0372
25,0.02,0.0248,0.0274,0.03,0.0346


In [14]:
using Serialization

serialize("chain-file.jls", chain_var)

In [12]:

b = rand(Geometric(1/66),2000)
quantile(b, [0, .25, .5, .75, 1])'


1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  18.75  45.0  90.0  617.0